<a href="https://colab.research.google.com/github/LudwigBitHoven/MillionAgents_testTask/blob/main/scrapers/testTask_4lapy_webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установим selenium

In [ ]:
!pip install selenium

# Импортируем и настроим джентльменский набор для скрапинга

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
# аргумент для работы с мобильной версией сайта
options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
options.add_argument("start-maximized")
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

# Посмотрим что на странице

Страница товара не имеет бесконечного скролла и товары распределены по отдельным url-ам вида



> "https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=1"


#### Можно захардкодить скрапируемые адреса с товарами следующим образом, меняя лишь параметр page:


In [ ]:
pages = []
for i in range(1, 10):
  pages.append(f"https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page={i}")
pages

['https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=1',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=2',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=3',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=4',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=5',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=6',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=7',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=8',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=9']

## А можно сделать лучше и спарсить количество страниц по указанному адресу категории товаров:

In [ ]:
# категория "сухой корм для кошек"
url = "https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/"
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)
# получаем все связанное с пагинацией
pagination = soup.findAll("li", class_="b-pagination__item")
pagination[6:]

[<li class="b-pagination__item">
 <a class="b-pagination__dot js-pagination" href="/catalog/koshki/korm-koshki/sukhoy/?page=7" title="7">
                             …
                         </a>
 </li>,
 <li class="b-pagination__item">
 <a class="b-pagination__link js-pagination" href="/catalog/koshki/korm-koshki/sukhoy/?page=9" title="9">
                         9                    </a>
 </li>,
 <li class="b-pagination__item b-pagination__item--next">
 <a class="b-pagination__link js-pagination" href="/catalog/koshki/korm-koshki/sukhoy/?page=2" title="2">
                     Вперед
                 </a>
 </li>]

In [ ]:
pagination[-2].find("a") # кнопка, ведущая на самую последнюю страницу в категории

<a class="b-pagination__link js-pagination" href="/catalog/koshki/korm-koshki/sukhoy/?page=9" title="9">
                        9                    </a>

In [ ]:
last_page = int(pagination[-2].find("a")["title"])
last_page # индекс последней страницы из имени кнопки

9

Теперь получим пул url адресов, с которых необходимо будет соскрапить товары

In [ ]:
pages = []
for i in range(1, last_page + 1):
  pages.append(f"https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page={i}")
pages

['https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=1',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=2',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=3',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=4',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=5',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=6',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=7',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=8',
 'https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=9']

## Попробуем вытянуть данные о товаре со страницы

Напишем функции, чтобы соскрапить json, содержащий информацию о бренде, со страницы и перевести в читаемый для Python формат

In [ ]:
def extract_json(s):
  for idx_beg, char in enumerate(s): # ищем индекс открывающей скобки
    if char == "{":
      break
  for idx_end, char in enumerate(s[::-1]): # ищем индекс закрывающей скобки
    if char == "}":
      break
  idx_end = len(s) - idx_end
  s = s[idx_beg:idx_end]
  s = s.replace("'",'"') # для каста в json заменяем кавычки
  return json.loads(s)

def get_brand(obj):
  product_json = extract_json(obj)
  # хардкодинг для получения бренда
  return product_json['ecommerce']['click']['products'][0]['brand']

In [ ]:
driver.get("https://4lapy.ru/catalog/koshki/korm-koshki/sukhoy/?section_id=3&sort=popular&page=9")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)

In [ ]:
temp = products[0].find(class_="b-common-item__image-link js-item-link")['onclick'] # json, содержащий данные о бренде
get_brand(temp) # проверим возвращается ли бренд

'Hills'

Найдем остальные необходимые данные товаров

In [ ]:
products = soup.findAll("div", class_="b-common-item b-common-item--catalog-item js-product-item init-product-info")
tup = (
  products[0]["data-productid"], # id продукта, альтернативно можно найти артикул продукта data-product-articul
  products[0].find(class_="b-common-item__image js-weight-img")["title"], # название продукта
  products[0].find(class_="b-common-item__image-link js-item-link")["href"], # ссылка на товар
  products[0].find(class_="b-common-item__prev-price js-sale-origin").text, # цена без скидки
  products[0].find(class_="js-price-block").text) # цена со скидкой
for line in tup:
  print(line)

31169
Science Plan Perfect Weight Сухой корм для взрослых кошек, склонных к избыточному весу, с курицей, 1,5 кг
https://4lapy.ru/catalog/koshki/korm-koshki/dieticheskiy/Korm_dlya_koshek_Hills_idelnyy_ves_s_kuritsey.html?offer=31170

3499


Выше видно, что товар не по акции имеет пустое поле "цены без скидки". Место реальной цены товара вне акции занимает цена товара "со скидкой", как бы то странно ни звучало. Чтобы избежать возможных казусов будем следовать следующему плану: проверяем товар на наличие "цены без скидки", если она есть и она не пустая, то сохраняем её и "цену со скидкой" как разные значения, в ином случае сохраняем "цену без скидки", как равную "цене по скидке".

# Скрапинг

Чтобы проверять товар на наличие, будем проверять существование класса "b-common-item__text" и значение текста внутри него:

In [ ]:
products = soup.findAll("div", class_="b-common-item b-common-item--catalog-item js-product-item init-product-info")
temp = products[0].find(class_="b-common-item__image-link js-item-link")['onclick'] # json, содержащий данные о бренде
get_brand(temp)
for product in products:
  if product.find(class_="b-common-item__text"): # проверка на существование
      if product.find(class_="b-common-item__text").text == "Нет в наличии": # проверка на текст
        print("product is out of stock")
        continue
  temp = product.find(class_="b-common-item__image-link js-item-link")['onclick']
  print(get_brand(temp))

Hills
Whiskas
Hills
Hills
PRO PLAN® Veterinary Diets
Hills
MEAT GARDEN
PRO PLAN®
Guabi Natural
Hills
UnoCat
Grandin
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock
product is out of stock


In [ ]:
data = {"product_id": [], "title": [], "link": [], "discount_price": [], "regular_price": [], "brand": []}

for page in pages:
  driver.get(page) # делаем запрос и получаем страницу
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # скроллим вниз для загрузки всех товаров
  time.sleep(5) # ждем прогрузки
  html = driver.page_source
  soup = BeautifulSoup(html)
  # составляем список товаров
  products = soup.findAll("div", class_="b-common-item b-common-item--catalog-item js-product-item init-product-info")
  for product in products:
    # проверка на наличие товара
    if product.find(class_="b-common-item__text"):
      if product.find(class_="b-common-item__text").text == "Нет в наличии":
        print("Product is out of stock")
        continue
    # добавляем id продукта
    data["product_id"].append(product["data-productid"])
    # добавляем название продукта
    data["title"].append(product.find(class_="b-common-item__image js-weight-img")["title"])
    # добавляем ссылку на продукт
    data["link"].append(product.find(class_="b-common-item__image-link js-item-link")["href"])
    # добавляем цену по скидке
    discount_price = int(product.find(class_="js-price-block").text)
    data["discount_price"].append(discount_price)
    # проверяем наличие цены без скидки по вышеописанной логике
    regular_price = product.find(class_="b-common-item__prev-price js-sale-origin").text
    if regular_price:
      regular_price = int(regular_price.split()[0])
    else:
      regular_price = discount_price
    # добавляем цену без скидки
    data["regular_price"].append(regular_price)
    # добавляем бренд
    temp = product.find(class_="b-common-item__image-link js-item-link")['onclick']
    data["brand"].append(get_brand(temp))
  print("Page scraped")

Page scraped
Page scraped
Page scraped
Page scraped
Page scraped
Page scraped
Page scraped
Product is out of stock
Page scraped
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Product is out of stock
Page scraped


# Сохраняем датафрейм

In [ ]:
df = pd.DataFrame.from_dict(data)
df

,product_id,title,link,discount_price,regular_price,brand
0,168899,"Сухой корм для стерилизованных кошек, с индейк...",https://4lapy.ru/catalog/koshki/korm-koshki/ko...,1199,1199,Wellkiss
1,168902,"Сухой корм для стерилизованных кошек, с ягненк...",https://4lapy.ru/catalog/koshki/korm-koshki/su...,1199,1199,Wellkiss
2,151160,Adult Sterilized Classic Сухой корм для стерил...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,2629,2629,Ownat
3,119899,N&D PRIME сухой корм для стерилизованных кошек...,https://4lapy.ru/catalog/koshki/korm-koshki/kh...,3683,3683,Farmina
4,151172,Adult Sterilized Корм сухой беззерновой для ст...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,3289,3289,Ownat
...,...,...,...,...,...,...
294,108461,Sterilised Сухой корм для взрослых стерилизова...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,629,629,PRO PLAN®
295,159391,Natural Cat Сухой корм для взрослых кошек с цы...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,1975,1975,Guabi Natural
296,80053,Science Plan Сухой корм для взрослых кошек для...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,4869,4869,Hills
297,135823,Корм сухой для взрослых привередливых кошек с ...,https://4lapy.ru/catalog/koshki/korm-koshki/su...,279,279,UnoCat


In [ ]:
df.to_csv("products.csv")